In [2]:
import glob
import pandas as pd
import numpy as np

In [8]:
def to_config(line,access):
    return eval(line.replace(" ".join(line.split(' ')[:4]),'').replace(f'%s Mismatches'%access,"{'%s'"%access)+"}")

def get_registers(idict):
    parameters = []
    for access,accessDict in idict.items():
        if 'ASIC' in accessDict.keys():
            accessDict = accessDict['ASIC']
        for block,blockDict in accessDict.items():
            for param, paramDict in blockDict.items():
                parameters.append(f'{access}_{block}_{param}')
    return parameters

            
RW_hardreset = {}
RO_hardreset = {}
with open('logs/Monitoring_logs/log_March_12_InBeam_ForcedReset.txt', 'r') as f:
    for line in f:
        if 'RW Mismatches' in line and not RW_hardreset:
            RW_hardreset = to_config(line,'RW')
        if 'RO Mismatches' in line and not RW_hardreset:
            RO_hardreset = to_config(line,'RO')
        
to_read = [
    'n_OOT_errors',
    'n_IT_errors',
    'n_RW_mismatches',
    'n_RO_mismatches',
    'hardresets',
    'RW_mismatches',
    'RO_mismatches',
    'RO_aligner_mismatches'
    'start_time',
    'stop_time',
    'start_time_OOT',
    'stop_time_OOT',
    'log_name',
    'netx',
    'algorithm',
]

logList=glob.glob('logs/Monitoring_logs/log_March_15_Afternoon.txt')
monitoring_logs = {}

from deepdiff import DeepDiff

for log in logList:
    file1 = open(log, 'r')
    lines = file1.readlines()
    key = ""
    dataset = ""
    netx = 13
    algo = 0
    for i,line in enumerate(lines):
        try:
            if 'csv file' in line:
                dataset = line.split(' ')[-1]
            elif 'Initial RW Status' in line:
                config = eval(lines[i+1].replace(" ".join(lines[i+1].split(' ')[:4]),''))
                netx = int(config['FMTBUF_ALL']['config_eporttx_numen'])
                algo = int(config['MFC_ALGORITHM_SEL_DENSITY']['algo_select'])
            elif 'Starting data comparisons' in line and 'PULSE IS COMING' in lines[i+1]:
                key = ' '.join(lines[i+1].split(' ')[:2]) # take timestamp from pulse
                monitoring_logs[key] = dict.fromkeys(to_read)
                monitoring_logs[key]['start_time'] = ' '.join(line.split(' ')[:2])
                monitoring_logs[key]['log_name'] = log
                monitoring_logs[key]['dataset'] = dataset.split('/')[-2]
                monitoring_logs[key]['netx'] = netx
                monitoring_logs[key]['algorithm'] = algo
            elif 'Stopping comparisons' in line and 'Error counter' in lines[i+1]:
                monitoring_logs[key]['stop_time'] = ' '.join(line.split(' ')[:2])
            elif 'Stopping OOT' in line and 'Error counter' in lines[i+1]:
                monitoring_logs[key]['n_OOT_errors'] = int(lines[i+1].split('\n')[0].split(' ')[-1])
                monitoring_logs[key]['stop_time_OOT'] = ' '.join(line.split(' ')[:2])
            elif 'Starting OOT' in line:
                monitoring_logs[key]['start_time_OOT'] = ' '.join(line.split(' ')[:2])
            elif 'Stopping comparisons' in line and 'Error counter' in lines[i+1]:
                monitoring_logs[key]['n_IT_errors'] = int(lines[i+1].split('\n')[0].split(' ')[-1])
            elif 'RW Mismatches' in line:
                config = to_config(line,'RW')
                reg = get_registers(config)
                monitoring_logs[key]['n_RW_mismatches'] = len(reg)
                if config==RW_hardreset:
                    monitoring_logs[key]['hardresets'] = 1
                    monitoring_logs[key]['RW_mismatches'] = []    
                else:
                    ddiff = DeepDiff(config, RW_hardreset, ignore_order=True)
                    if 'iterable_item_removed' not in ddiff.keys():
                        monitoring_logs[key]['hardresets'] = 1
                        monitoring_logs[key]['RW_mismatches'] = [] 
                    else:
                        monitoring_logs[key]['hardresets'] = None
                        monitoring_logs[key]['RW_mismatches'] = reg
            elif 'ERROR  RO Mismatches' in line:
                config = get_registers(to_config(line,'RO'))
                monitoring_logs[key]['n_RO_mismatches'] = len(config)
                if monitoring_logs[key]['hardresets'] == 1:
                    monitoring_logs[key]['RO_mismatches'] = []
                else:
                    monitoring_logs[key]['RO_mismatches'] = config
                #print(key,monitoring_logs[key]['RO_mismatches'])
            elif 'DEBUG  RO Mismatches' in line:
                config = get_registers(to_config(line,'RO'))
                if monitoring_logs[key]['hardresets'] == 1:
                    monitoring_logs[key]['RO_aligner_mismatches'] = []
                else:
                    monitoring_logs[key]['RO_aligner_mismatches'] = config
        except:
            print('Could not parse ',log)
            break
            
config = {
    'PulseTime': [],
    'StartTime': [],
    'StopTime': [],
    'StartTimeOOT': [],
    'StopTimeOOT': [], 
    'nOOT': [],
    'nIT': [],
    'HardReset': [],
    'nRW': [],
    'nRO': [],
    'RW': [],
    'RO': [],
    'ROaligner': [],
    'Log': [],
    'Dataset': [],
    'neTx': [],
    'Algo': [],
}
for key,item in monitoring_logs.items():
    config['PulseTime'].append(key)
    config['StartTime'].append(item['start_time'])
    config['StopTime'].append(item['stop_time'])
    config['StartTimeOOT'].append(item['start_time_OOT'])
    config['StopTimeOOT'].append(item['stop_time_OOT'])
    config['HardReset'].append(item['hardresets'] if item['hardresets'] else 0)
    config['nRW'].append(item['n_RW_mismatches'] if item['n_RW_mismatches'] else 0)
    config['nRO'].append(item['n_RO_mismatches'] if item['n_RO_mismatches'] else 0)
    config['nOOT'].append(item['n_OOT_errors'] if item['n_OOT_errors'] else 0)
    config['nIT'].append(item['n_IT_errors'] if item['n_IT_errors'] else 0)
    config['RW'].append(item['RW_mismatches'] if item['RW_mismatches'] else [])
    config['RO'].append(item['RO_mismatches'] if item['RO_mismatches'] else [])
    config['ROaligner'].append(item['RO_aligner_mismatches'] if 'RO_aligner_mismatches' in item else [])
    config['Log'].append(item['log_name'])
    config['Dataset'].append(item['dataset'])
    config['neTx'].append(item['netx'])
    config['Algo'].append(item['algorithm'])

df=pd.DataFrame.from_dict(config)
df.set_index('PulseTime',inplace=True)
df.sort_index(inplace=True)

In [9]:
df

,StartTime,StopTime,StartTimeOOT,StopTimeOOT,nOOT,nIT,HardReset,nRW,nRO,RW,RO,ROaligner,Log,Dataset,neTx,Algo
PulseTime,,,,,,,,,,,,,,,,
03-15-22 12:51:25,03-15-22 12:51:20,03-15-22 12:51:30,03-15-22 12:51:50,03-15-22 12:52:00,0,0,0,0,0,[],[],[],logs/Monitoring_logs/log_March_15_Afternoon.txt,counterPatternInTC,13,3
03-15-22 12:52:26,03-15-22 12:52:21,03-15-22 12:52:31,03-15-22 12:52:50,03-15-22 12:53:00,0,0,0,0,0,[],[],[],logs/Monitoring_logs/log_March_15_Afternoon.txt,counterPatternInTC,13,3
03-15-22 12:53:25,03-15-22 12:53:20,03-15-22 12:53:30,03-15-22 12:53:51,03-15-22 12:54:01,0,0,0,0,0,[],[],[],logs/Monitoring_logs/log_March_15_Afternoon.txt,counterPatternInTC,13,3
03-15-22 12:54:25,03-15-22 12:54:20,03-15-22 12:54:30,03-15-22 12:54:50,03-15-22 12:55:00,0,0,0,0,0,[],[],[],logs/Monitoring_logs/log_March_15_Afternoon.txt,counterPatternInTC,13,3
03-15-22 12:55:25,03-15-22 12:55:20,03-15-22 12:55:30,03-15-22 12:55:50,03-15-22 12:56:00,0,0,0,0,0,[],[],[],logs/Monitoring_logs/log_March_15_Afternoon.txt,counterPatternInTC,13,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
03-15-22 18:50:15,03-15-22 18:50:10,03-15-22 18:50:20,03-15-22 18:50:40,03-15-22 18:50:51,0,0,0,0,0,[],[],[],logs/Monitoring_logs/log_March_15_Afternoon.txt,counterPatternInTC,13,3
03-15-22 18:51:15,03-15-22 18:51:10,03-15-22 18:51:20,03-15-22 18:51:40,03-15-22 18:51:50,0,0,0,0,0,[],[],[],logs/Monitoring_logs/log_March_15_Afternoon.txt,counterPatternInTC,13,3
03-15-22 18:52:16,03-15-22 18:52:11,03-15-22 18:52:21,03-15-22 18:52:40,03-15-22 18:52:50,0,0,0,0,0,[],[],[],logs/Monitoring_logs/log_March_15_Afternoon.txt,counterPatternInTC,13,3


In [10]:
df.to_csv('logs/MonitoringInfo.csv')

In [51]:
for key in df.index:
    if '03-11-22 17:38' in key:
        print(key)


03-11-22 17:38:51


In [52]:
df_error=pd.read_csv('logs/ErrorInfo.csv')
df_error.set_index('Time',inplace=True)
df_error

,fName_ASIC,fName_Emulator,fName_Input,RelPosition,MotionTable,Errors,logFile
Time,,,,,,,
2022-03-11 17:38:56,logs/DAQ_logs/lc-ASIC_11mar_173856.csv,logs/DAQ_logs/lc-emulator_11mar_173856.csv,logs/DAQ_logs/lc-input_11mar_173856.csv,25,269,92955406,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
2022-03-11 17:39:26,logs/DAQ_logs/lc-ASIC_oot_11mar_173926.csv,logs/DAQ_logs/lc-emulator_oot_11mar_173926.csv,logs/DAQ_logs/lc-input_oot_11mar_173926.csv,25,269,360536469,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
2022-03-11 17:39:56,logs/DAQ_logs/lc-ASIC_11mar_173956.csv,logs/DAQ_logs/lc-emulator_11mar_173956.csv,logs/DAQ_logs/lc-input_11mar_173956.csv,25,269,400565817,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
2022-03-11 17:40:26,logs/DAQ_logs/lc-ASIC_oot_11mar_174026.csv,logs/DAQ_logs/lc-emulator_oot_11mar_174026.csv,logs/DAQ_logs/lc-input_oot_11mar_174026.csv,25,269,400618586,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
2022-03-11 17:40:57,logs/DAQ_logs/lc-ASIC_11mar_174057.csv,logs/DAQ_logs/lc-emulator_11mar_174057.csv,logs/DAQ_logs/lc-input_11mar_174057.csv,25,269,400636805,logs/Monitoring_logs/log_March_11_Beam_Pos_269...
...,...,...,...,...,...,...,...
2022-03-17 17:03:56,logs/DAQ_logs/lc-ASIC_tsalgo_randdata_17mar_17...,logs/DAQ_logs/lc-emulator_tsalgo_randdata_17ma...,logs/DAQ_logs/lc-input_tsalgo_randdata_17mar_1...,25,269,28,logs/Monitoring_logs/log_March_17_TS_Test_Rand...
2022-03-17 17:04:27,logs/DAQ_logs/lc-ASIC_tsalgo_randdata_oot_17ma...,logs/DAQ_logs/lc-emulator_tsalgo_randdata_oot_...,logs/DAQ_logs/lc-input_tsalgo_randdata_oot_17m...,25,269,3,logs/Monitoring_logs/log_March_17_TS_Test_Rand...
2022-03-17 17:04:57,logs/DAQ_logs/lc-ASIC_tsalgo_randdata_17mar_17...,logs/DAQ_logs/lc-emulator_tsalgo_randdata_17ma...,logs/DAQ_logs/lc-input_tsalgo_randdata_17mar_1...,25,269,34,logs/Monitoring_logs/log_March_17_TS_Test_Rand...
